# Create a deep dream

https://keras.io/examples/deep_dream/


In [1]:
import os
from PIL import Image
from keras.applications.vgg16 import VGG16
from keras.applications import inception_v3
import numpy as np
import keras.backend as K
from keras.models import Model
from keras.utils import plot_model
from keras.preprocessing.image import load_img, save_img, img_to_array
import scipy

Using TensorFlow backend.


## Load the input

In [2]:
inputPic = "D:/Project/DeepLearning/DeepDream/Data/original/IMG_20170509_155330.jpg"

In [3]:
def preprocess_image(image_path):
    '''Load the picture; convert it as numpy array'''
    img = load_img(image_path)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [4]:
img = preprocess_image(inputPic)

## Model creation

In [5]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=img[0].shape )
dream = model.input

In [6]:
K.set_learning_phase(0)

**Create 5 amplifications:**
* features 0 : The values given by keras
* features 1 Amplification of first layers' activation, and reduce other.
* features 2 Amplification of middle layers activation
* features 3 Amplified the layers in the end of inception
* features 4 Amplified all the activations

In [7]:
# Parametres
step = 0.01  # Gradient ascent step size
num_octave = 5  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 20  # Number of ascent steps per scale
max_loss = 10.

In [8]:
#Dictionnary of the layers to amplified + weights
settings = {'features0': {'mixed2': 0.2,'mixed3': 0.5,'mixed4': 2.,'mixed5': 1.5,},
           'features1': {'mixed2': 4.,'mixed3': 3.5,'mixed4': .02,'mixed5': .005,},
           'features2': {'mixed2': .1,'mixed3': 5.,'mixed4': 2.,'mixed5': .1,},
            'features3': {'mixed2': 0.02,'mixed3': 0.1,'mixed4': 3.,'mixed5': 5.,},
            'features4': {'mixed2': 3.,'mixed3': 3.,'mixed4': 3.,'mixed5': 3.,},
           }

In [9]:
#Create a dictionnary with coupple layer's name, layer's object
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [10]:

for key in settings.keys():

    #Loss function
    loss = K.variable(0.)
    for layer_name in settings[key]:

        if layer_name not in layer_dict:
            raise ValueError('Layer ' + layer_name + ' not found in model.')


        coeff = settings[key][layer_name] # Get the coefficient from setting dict for a layer
        x = layer_dict[layer_name].output # Get the activation values

        scaling = K.prod(K.cast(K.shape(x), 'float32')) # Number of neurons in the layer
        if K.image_data_format() == 'channels_first':
            loss = loss + coeff * K.sum(K.square(x[:, :, 2: -2, 2: -2])) / scaling
        else:
            #The loss is sum of MSE of activation value 
            loss = loss + coeff * K.sum(K.square(x[:, 2: -2, 2: -2, :])) / scaling

    #Gradient
    grads = K.gradients(loss, dream)[0]
    # Normalize gradients.
    grads /= K.maximum(K.mean(K.abs(grads)), K.epsilon())
    outputs = [loss, grads]
    fetch_loss_and_grads = K.function([dream], outputs)

    def eval_loss_and_grads(x):
        '''Loss and gradient evaluation'''
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1]
        return loss_value, grad_values

    def resize_img(img, size):
        '''Return a zommed version of the image'''
        img = np.copy(img)
        if K.image_data_format() == 'channels_first':
            factors = (1, 1,
                       float(size[0]) / img.shape[2],
                       float(size[1]) / img.shape[3])
        else:
            factors = (1,
                       float(size[0]) / img.shape[1],
                       float(size[1]) / img.shape[2],
                       1)
        return scipy.ndimage.zoom(img, factors, order=1)


    def gradient_ascent(x, iterations, step, max_loss=None):
        ''''''
        for i in range(iterations):
            loss_value, grad_values = eval_loss_and_grads(x)
            if max_loss is not None and loss_value > max_loss:
                break
            print('..Loss value at', i, ':', loss_value)
            x += step * grad_values
        return x


    if K.image_data_format() == 'channels_first':
        original_shape = img.shape[2:]
    else:
        original_shape = img.shape[1:3]

    successive_shapes = [original_shape]

    for i in range(1, num_octave):
        shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
        successive_shapes.append(shape)

    successive_shapes = successive_shapes[::-1]
    original_img = np.copy(img)
    shrunk_original_img = resize_img(img, successive_shapes[0])

    for shape in successive_shapes:
        print('Processing image shape', shape)
        img = resize_img(img, shape)
        img = gradient_ascent(img,
                              iterations=iterations,
                              step=step,
                              max_loss=max_loss)
        upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
        same_size_original = resize_img(original_img, shape)
        lost_detail = same_size_original - upscaled_shrunk_original_img

        img += lost_detail
        shrunk_original_img = resize_img(original_img, shape)


    save_img("D:/Project/DeepLearning/DeepDream/Output/inception/flower/res_" + key + '.png', deprocess_image(np.copy(img)))

    img = preprocess_image(inputPic)


Processing image shape (266, 149)
..Loss value at 0 : 0.68362355
..Loss value at 1 : 0.81318736
..Loss value at 2 : 1.1164751
..Loss value at 3 : 1.4358656
..Loss value at 4 : 1.7089396
..Loss value at 5 : 1.9051473
..Loss value at 6 : 2.125979
..Loss value at 7 : 2.3484793
..Loss value at 8 : 2.5651379
..Loss value at 9 : 2.7165856
..Loss value at 10 : 2.918583
..Loss value at 11 : 3.082869
..Loss value at 12 : 3.310677
..Loss value at 13 : 3.4231477
..Loss value at 14 : 3.6136048
..Loss value at 15 : 3.6865506
..Loss value at 16 : 3.895085
..Loss value at 17 : 4.0061727
..Loss value at 18 : 4.1812334
..Loss value at 19 : 4.2809973
Processing image shape (373, 209)
..Loss value at 0 : 1.1967751
..Loss value at 1 : 1.7420886
..Loss value at 2 : 2.1521103
..Loss value at 3 : 2.4658298
..Loss value at 4 : 2.8205535
..Loss value at 5 : 3.1059904
..Loss value at 6 : 3.4135957
..Loss value at 7 : 3.7045114
..Loss value at 8 : 3.9644113
..Loss value at 9 : 4.239211
..Loss value at 10 : 4.494

In [11]:

img = [inputPic]
for i in settings.keys():
    img.append("D:/Project/DeepLearning/DeepDream/Output/inception/flower/res_{}.png".format(i))


img = list(map(lambda s: img_to_array(load_img(s)   ) , img )  )

r = img[0]
for i in img[1:]:
    r = np.concatenate([r,i], axis =1)

Image.fromarray(r.astype('uint8')).save("D:/Project/DeepLearning/DeepDream/Output/inception/flower/comparison.png")